In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import requests
import yfinance as yf
import datetime
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from regressors import stats
import bt

In [23]:
test = yf.download(['MSFT'])

[*********************100%***********************]  1 of 1 completed


In [27]:
test = test.reset_index()

In [29]:
test['Ticker'] = 'MSFT'

In [32]:
test['Date'].iloc[0]

Timestamp('1986-03-13 00:00:00')

In [35]:
test['ts'] = pd.Timestamp(test['Date'].iloc[0]) + pd.Timedelta('16H')

In [38]:
test.rename(columns={'Date':'asof_date','Open':'open','Close':'close','High':'high','Low':'low','Volume':'volume',
                     'Ticker':'ticker','Adj Close':'adj_close'})

,asof_date,open,high,low,close,adj_close,volume,ticker,ts
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.061491,1031788800,MSFT,1986-03-13 16:00:00
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.063687,308160000,MSFT,1986-03-13 16:00:00
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.064785,133171200,MSFT,1986-03-13 16:00:00
3,1986-03-18,0.102431,0.103299,0.098958,0.099826,0.063138,67766400,MSFT,1986-03-13 16:00:00
4,1986-03-19,0.099826,0.100694,0.097222,0.098090,0.062040,47894400,MSFT,1986-03-13 16:00:00
...,...,...,...,...,...,...,...,...,...
8988,2021-11-08,337.299988,337.649994,334.440002,336.989990,336.989990,20897000,MSFT,1986-03-13 16:00:00
8989,2021-11-09,337.109985,338.720001,334.529999,335.950012,335.950012,21307400,MSFT,1986-03-13 16:00:00
8990,2021-11-10,334.570007,334.630005,329.920013,330.799988,330.799988,25500900,MSFT,1986-03-13 16:00:00
8991,2021-11-11,331.250000,333.769989,330.510010,332.429993,332.429993,16849800,MSFT,1986-03-13 16:00:00


In [21]:
POSTGRES = {
'user' : 'postgres',
'pw' : 'Everglade159753',
'host' : '34.142.124.233',
'port' : '5432',
'db' : 'postgres'
}


import sqlalchemy
import pandas as pd
import os
import yfinance as yf

class Config(object):
    # ...
    SQLALCHEMY_DATABASE_URI =  os.environ.get('DATABASE_URL') or 'postgresql://%(user)s:%(pw)s@%(host)s:%(port)s/%(db)s' % POSTGRES
    SQLALCHEMY_TRACK_MODIFICATIONS = False

config = Config()

cnx = sqlalchemy.create_engine(config.SQLALCHEMY_DATABASE_URI)

In [22]:
df = pd.read_sql_query('SELECT * FROM stocksPrice;', cnx)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "34.142.124.233" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [ ]:
df

### Get all recent tickers of S&P500

In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]

In [ ]:
len(tickers)

In [ ]:
data = yf.download(["SPY"]+tickers, period = "5y")

### Compute returns

In [ ]:
rets = data['Adj Close'].pct_change()
rets.dropna(inplace=True,axis=0, how='all')
rets.dropna(inplace=True,axis=1, how='any')

rets_SPX = pd.DataFrame(rets['SPY'])
rets.drop(['SPY'], axis=1, inplace=True)

In [ ]:
mu = rets.mean()
sig = ret.std()

z_rets = (rets - mu)/sig

In [ ]:
(1+ret_SPX).cumprod().plot()

In [ ]:
z_rets['AAPL'].plot.hist(bins=20)

In [ ]:
pca = PCA(n_components=5).fit(rets)

In [ ]:
pcs = ['PC' + str(i) for i in range(1,51)]
explained_variance_ratio = pd.DataFrame(zip(pcs, pca.explained_variance_ratio_), columns=['PC', 'RatioVarianceExplained'])


In [ ]:
sns.barplot(data=explained_variance_ratio, x='PC', y='RatioVarianceExplained')

### Eigenportfolio returns

In [ ]:
w=abs(pca.components_[0])/sum(abs(pca.components_[0]))
eigenptf = pd.DataFrame(rets.dot(w.T), columns=['Returns'])
eigenptf['CumRets'] = (1+eigenptf['Returns']).cumprod()
eigenptf = eigenptf.join(rets_SPX, how='left')
eigenptf['SPYCumRets'] = (1+eigenptf['SPY']).cumprod()
eigenptf[['CumRets','SPYCumRets']].plot()

In [ ]:
pd.DataFrame(zip(rets.columns, pca.components_[0]), columns=['Tickers', 'Eigenvector']).plot(x='Tickers')

### Getting sectors ETF data

In [ ]:
etf_tickers = ['QQQ','IYR','IYT','OIH','KBE','RTH','SMH','UTH','XLE','XLF','XLI','XLK','XLP','XLV']
etf_data = yf.download(etf_tickers, period = "5y")

In [ ]:
etf_data['Adj Close'].dropna()

In [ ]:
rets_etf = etf_data['Adj Close'].pct_change()
rets_etf = rets_etf.dropna()

In [ ]:
rets_etf.shape

In [ ]:
rets.shape

In [ ]:
df = rets[['JPM']].join(rets_etf, how='left')

In [ ]:
corrMat = df.corr()
sns.heatmap(corrMat)

## JPM vs Financials ETF

### OLS vs Ridge Regression

In [ ]:
model = sm.OLS(rets['JPM'],rets_etf['XLF'])
res = model.fit()
print(res.summary())

beta_etf = res.params[0]

In [ ]:
model = Ridge(alpha=0.01).fit(rets_etf[etf_tickers],rets['JPM'].values)

In [ ]:
summary = stats.summary(model,rets_etf[etf_tickers],rets['JPM'].values)

In [ ]:
res.resid.plot()

### Modelling errors as an OU process

#### Using a 60-days window as in Avallaneda et al. 2019

In [ ]:
X_process= res.resid.rolling(60).sum()
X_process.plot()

#### Estimate AR(1) parameters

$dX_t = \kappa(m-X_{t-1})dt + \sigma dW_t$

Then we can write:

$X_t = a + bX_{t-1} + \epsilon_t$

with:

$a = (1 - e^{-\kappa \Delta t})m$

$b =  e^{-\kappa \Delta t}$

$V(\epsilon) = \sigma^2 \frac{1-e^{-2 \kappa \Delta t}}{2 \kappa}$

In [ ]:
res_df = pd.DataFrame(X_process, columns=['X'])
res_df['lagged_X'] = res_df['X'].shift()
res_df.dropna(inplace=True)

In [ ]:
X = res_df['lagged_X']
X = sm.add_constant(X)
model = sm.OLS(res_df['X'],X)
res = model.fit()
print(res.summary())

In [ ]:
kappa = -np.log(res.params[1]) * 252
m = res.params[0]/(1-res.params[1])
var_eps = np.var(res.resid)
sig_X = np.sqrt(var_eps*2*kappa/(1-res.params[1]**2))
sig_eq = np.sqrt(var_eps/(1-res.params[1]**2))

### Build S-score: measure of distance to equilibrium of the cointegrated residual in units standard deviations

In [ ]:
res_df['s_score'] = (res_df['X'] - m)/sig_eq

In [ ]:
res_df['s_score'].plot()

In [ ]:
tw = res_df[['s_score']].copy()

In [ ]:
sbo, sso = 1.25,1.25
sbc = 0.75
ssc = 0.50

tw['stock_weights'] = tw['s_score']*0
tw['etf_weights'] = tw['s_score']*0

tw['stock_weights'].iloc[np.where(tw['s_score'] < - sbo)] = 1
tw['etf_weights'].iloc[np.where(tw['s_score'] < - sbo)] = - beta_etf
tw['stock_weights'].iloc[np.where(tw['s_score'] > sso)] = -1
tw['etf_weights'].iloc[np.where(tw['s_score'] > sso)] = + beta_etf

# tw['weights'].iloc[np.where(tw['s_score'] < sbc)] = 0
# tw['weights'].iloc[np.where(tw['s_score'] > - ssc)] = 0

In [ ]:
res_df['s_score']

In [ ]:
sbo, sso = 1.25,1.25
sbc = 0.75
ssc = 0.50

res_df['stock_weights'] = res_df['s_score'] * 0.0

for t in range(1, res_df['s_score'].shape[0]):
    if res_df['stock_weights'].iloc[t-1]==0.0:
        if res_df['s_score'].iloc[t] < -sbo:
            res_df['stock_weights'].iloc[t] = 1.0
        elif res_df['s_score'].iloc[t] > sso:
            res_df['stock_weights'].iloc[t] = -1.0
        else:
            res_df['stock_weights'].iloc[t] = 0.0
    else:
        if res_df['stock_weights'].iloc[t-1] < 0 and res_df['s_score'].iloc[t] < sbc:
            res_df['stock_weights'].iloc[t] = 0.0
        elif res_df['stock_weights'].iloc[t-1] > 0 and res_df['s_score'].iloc[t]> -ssc:
            res_df['stock_weights'].iloc[t] = 0.0
        else:
            res_df['stock_weights'].iloc[t] = res_df['stock_weights'].iloc[t-1]



In [ ]:
res_df['etf_weights'] = - beta_etf * res_df['stock_weights']

In [ ]:
res_df[['stock_weights','etf_weights', 's_score']].loc['2017-05-31':'2017-06-05']

In [ ]:
tw = res_df[['stock_weights','etf_weights']].rename(columns={'stock_weights':'JPM', 'etf_weights':'XLF'})

In [ ]:
tw.describe()

In [ ]:
price_data = pd.DataFrame(data['Adj Close']['JPM']).join(etf_data['Adj Close']['XLF'], how='left')

In [ ]:
s = bt.Strategy('etf_vs_stock', [bt.algos.WeighTarget(tw),bt.algos.Rebalance()],)

# now we create the Backtest
t = bt.Backtest(s, price_data)

In [ ]:
res = bt.run(t)

In [ ]:
res.display()

In [ ]:
res.plot()

In [ ]:
wdf = res.get_security_weights()
wdf.rename(columns={'JPM':'w_stock', 'XLF':'w_etf'},inplace=True)

In [ ]:
wdf.join(res_df['s_score'], how='left').join(price_data, how='left').to_excel('weights.xlsx')

## Scanning multiple stocks

In [ ]:
sbux = yf.Ticker("SBUX")
tlry = yf.Ticker("TLRY")

print(sbux.info['sector'])
print(tlry.info['sector'])

In [ ]:
ticks = yf.Tickers(tickers)

In [ ]:
sector = []
for tick in tickers:
    try:
#         sector.append(yf.Ticker(tick).info['sector'])
    except:
        print('Error with ticker: ' + tick)

In [ ]:
np.unique(sector)

In [ ]:
yf.Ticker(ticks.symbols[0]).info['shortName']

In [ ]:
yf.Ticker(etf_tickers[0]).info

In [ ]:
for tick in etf_tickers:
    print(tick, " :", yf.Ticker(tick).info['shortName'])

In [ ]:
sector_etf_tick = {'IYR':'Real Estate', 'IYT':'Transportation', 'OIH':'Oil Services', 'KBE': 'Banks', 
                   'RTH':'Retail', 'XLE':'Energy', 'XLF':'Financials', 'XLI':'Industrials', 'XLK':'Technology', 
                   'XLP':'Consumer Cyclical', 'XLV':'Healthcare'}

In [ ]:
def get_beta(stock, etf, window=60):
    # make sure indices are aligned
    etf = etf.loc[stock.index]
    
    # run regression and return: beta, p-val, residuals
    model = sm.OLS(stock,etf)
    res = model.fit()
    
    return res.params[0], res.pvalue[0], res.resid

In [ ]:
def OH_params_estimation(resid, window=60):
    X = resid.rolling(window).sum()
    df = pd.DataFrame(X, columns=['X'])
    df['lagged_X'] = res_df['X'].shift()
    df.dropna(inplace=True)
    
    y = df['X']
    lagged_X = df['lagged_X']
    lagged_X = sm.add_constant(lagged_X)
    model = sm.OLS(y,lagged_X)
    res = model.fit()
    
    kappa = -np.log(res.params[1]) * 252
    m = res.params[0]/(1-res.params[1])
    var_eps = np.var(res.resid)
    sig_X = np.sqrt(var_eps*2*kappa/(1-res.params[1]**2))
    sig_eq = np.sqrt(var_eps/(1-res.params[1]**2))
    
    return kappa, m, var_eps, sig_X, sig_eq